In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import json
from cpmpy import * # pip3 install cpmpy
from cpmpy.solvers import CPM_ortools

In [3]:
# Notebook to make magic square problems/instance

In [4]:
# latin square has rows/cols permutations (alldifferent)
def latin_sq(square):
        return [[AllDifferent(row) for row in square],
                [AllDifferent(col) for col in square.T]]

In [7]:
def model_latin_sq(N):
    square = intvar(1,N, shape=(N,N))
    return square, Model(latin_sq(square))
(square,m) = model_latin_sq(4)
m

Constraints:
    [alldifferent(IV16,IV17,IV18,IV19), alldifferent(IV20,IV21,IV22,IV23), alldifferent(IV24,IV25,IV26,IV27), alldifferent(IV28,IV29,IV30,IV31)]
    [alldifferent(IV16,IV20,IV24,IV28), alldifferent(IV17,IV21,IV25,IV29), alldifferent(IV18,IV22,IV26,IV30), alldifferent(IV19,IV23,IV27,IV31)]
Objective: None

In [8]:
m.solve()
print(square.value())

[[2 3 4 1]
 [1 4 2 3]
 [4 1 3 2]
 [3 2 1 4]]


In [9]:
m.solveAll()

576

In [10]:
N=4
np.random.randint(1,N+1, size=(N,N))

array([[2, 3, 1, 4],
       [4, 3, 1, 3],
       [1, 3, 3, 3],
       [4, 1, 1, 2]])

In [13]:
def make_inst(N, i, pos=200, neg=200):
    binData = dict()
    binData['problemType'] = 'type01'
    binData['instance'] = i
    binData['size'] = N
    binData['formatTemplate'] = dict()
    # N*N boolvars
    binData['formatTemplate']['square'] = [[dict([('high',N), ('low',1), ('type','dvar')])]*N ]*N
    binData['tests'] = []
    
    binData['solutions'] = []
    binData['nonSolutions'] = []
    
    # nonSolutions, uniformly
    (square,m) = model_latin_sq(N)
    while len(binData['nonSolutions']) < neg:
        s = SolverLookup.get("ortools", m)
        # get random assignment
        rand = np.random.randint(1,N+1, size=(N,N))
        s += [square == rand]
        # I make the same error here, that they need not be unique...
        if s.solve():
            binData['solutions'].append(dict([('square', rand.tolist())]))
        else:
            binData['nonSolutions'].append(dict([('square', rand.tolist())]))

    # solutions, uniformly guided, unique
    s = SolverLookup.get("ortools", m)
    while len(binData['solutions']) < pos:
        # get random assignment
        rand = np.random.randint(1,N+1, size=(N,N))
        s.solution_hint(square.flatten(), rand.flatten())
        s.solve()
        binData['solutions'].append(dict([('square', square.value().tolist())]))
        s += any(square != rand)
    
    return binData

N=4
make_inst(N, 0, N*1/4, N*1/4)

{'problemType': 'type01',
 'instance': 0,
 'size': 4,
 'formatTemplate': {'square': [[{'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'}],
   [{'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'}],
   [{'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'}],
   [{'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'},
    {'high': 4, 'low': 1, 'type': 'dvar'}]]},
 'tests': [],
 'solutions': [{'square': [[2, 3, 4, 1],
    [1, 4, 2, 3],
    [4, 1, 3, 2],
    [3, 2, 1, 4]]}],
 'nonSolutions': [{'square': [[4, 3, 4, 4],
    [1, 2, 2, 2],
    [1, 2, 1, 1],
    [1, 2, 1, 1]]}]}

In [10]:
# make multiple
for i,N in enumerate([4,5,6,7,8,9]):
    print(i,N)
    outdata = make_inst(N, i, N*50, N*50)
    json.dump(outdata, fp=open(f"instances/type21/instance{i}.json", 'w'))

0 4
1 5
2 6
3 7


^C pressed 1 times. Interrupting the solver. Press 3 times to force termination.


In [15]:
data = json.load(open('instances/type02/instance0.json'))
data

{'instance': 0,
 'tests': [],
 'size': 4,
 'formatTemplate': {'list': [{'high': 4, 'low': 1, 'type': 'dvar'},
   {'high': 4, 'low': 1, 'type': 'dvar'},
   {'high': 4, 'low': 1, 'type': 'dvar'},
   {'high': 4, 'low': 1, 'type': 'dvar'}]},
 'solutions': [{'list': [3, 1, 4, 2]}, {'list': [2, 4, 1, 3]}],
 'nonSolutions': [{'list': [2, 2, 1, 1]},
  {'list': [2, 1, 3, 2]},
  {'list': [4, 3, 2, 3]},
  {'list': [2, 3, 1, 1]},
  {'list': [1, 3, 4, 2]},
  {'list': [2, 3, 3, 4]},
  {'list': [2, 2, 4, 4]},
  {'list': [2, 2, 1, 3]},
  {'list': [1, 4, 4, 4]},
  {'list': [2, 3, 3, 3]},
  {'list': [4, 2, 2, 4]},
  {'list': [3, 2, 3, 3]},
  {'list': [2, 4, 3, 1]},
  {'list': [3, 1, 2, 4]},
  {'list': [3, 3, 3, 1]},
  {'list': [1, 4, 1, 3]},
  {'list': [4, 3, 1, 3]},
  {'list': [3, 1, 3, 4]},
  {'list': [2, 4, 4, 3]},
  {'list': [3, 1, 1, 4]},
  {'list': [3, 4, 4, 2]},
  {'list': [1, 3, 1, 1]},
  {'list': [2, 4, 3, 2]},
  {'list': [1, 2, 1, 2]},
  {'list': [4, 1, 3, 2]},
  {'list': [3, 3, 1, 3]},
  {'li

In [23]:
binData

{'instance': 0,
 'size': 4,
 'formatTemplate': {'board': [[{'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'}],
   [{'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'}],
   [{'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'}],
   [{'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'},
    {'high': 1, 'low': 0, 'type': 'dvar'}]]},
 'solutions': [{'board': [[0, 0, 1, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]]},
  {'board': [[0, 1, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0]]}],
 'nonSolutions': [{'board': [[0, 1, 0, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    

In [30]:
# convert all from 02 to 20
for i in range(10):
    indata = json.load(open(f"instances/type02/instance{i}.json"))
    outdata = convert_inst(indata)
    json.dump(outdata, fp=open(f"instances/type20/instance{i}.json", 'w'))